In [12]:
#pg2
import pandas as pd
from sqlalchemy import create_engine

def get_engine():
    user = "studente_dapt"
    password = "Ep1c0d3!!D4t4**4n4lys1s"
    host = "epicode-data-pt-mysql.cvetyjye2qbl.eu-central-1.rds.amazonaws.com"
    port = 3306
    db   = "AdventureWorksDW"

    uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db}"
    engine = create_engine(uri, echo=False)   # echo=False disables verbose SQL logging
    return engine

engine = get_engine()          # keep the engine alive for the whole cell

query = "SELECT * FROM dimproduct"
df = pd.read_sql(query, engine)

print("🔹 Dati originali")
print(df.head())

# a) Arrotonda a 2 cifre decimali
df["DealerPrice"] = df["DealerPrice"].round(2)

# b) Arrotonda al valore intero più vicino
df["DealerPrice"] = df["DealerPrice"].round(0).astype(int)

# c) Clip dei valori: min = 0, max = 1000
df["DealerPrice"] = df["DealerPrice"].clip(lower=0, upper=1000)

print("\n🔹 Dopo arrotondamento + clipping")
print(df.head())


# -------------------------------------------------
# 1️⃣  Creazione del DataFrame sintetico
# -------------------------------------------------
years = 5                                 # numero di anni da generare
guadagni = pd.DataFrame({
    "Mese":  list("GFMAMGLASOND" * years),
    "Anno":  np.repeat(list(range(years)), 12),
    "Valore": np.random.randint(800, 5000, 12 * years)
})

print("🔹 DataFrame di partenza")
print(guadagni.head(15))

# Aggiungiamo una colonna con la somma cumulativa su tutti i record
guadagni["Cumulativo_Totale"] = guadagni["Valore"].cumsum()

print("\n🔹 Dopo cumsum globale")
print(guadagni.head(15))

guadagni = guadagni.sort_values(["Anno", "Mese"]).reset_index(drop=True)
guadagni["Cumulativo_Totale"] = guadagni["Valore"].cumsum()

# Raggruppiamo per 'Anno' e calcoliamo la cumsum all’interno di ogni gruppo
guadagni["Cumulativo_Per_Anno"] = (
    guadagni.groupby("Anno")["Valore"]
          .cumsum()
)

print("\n🔹 Dopo cumsum per anno")

print(guadagni.head(15))

🔹 Dati originali
   ProductKey ProductAlternateKey  ProductSubcategoryKey  \
0           1             AR-5381                    NaN   
1           2             BA-8327                    NaN   
2           3             BE-2349                    NaN   
3           4             BE-2908                    NaN   
4           5             BL-2036                    NaN   

  WeightUnitMeasureCode SizeUnitMeasureCode     EnglishProductName  \
0                  None                None        Adjustable Race   
1                  None                None           Bearing Ball   
2                  None                None        BB Ball Bearing   
3                  None                None  Headset Ball Bearings   
4                  None                None                  Blade   

  SpanishProductName FrenchProductName  StandardCost  FinishedGoodsFlag  ...  \
0                                                NaN                  0  ...   
1                                        

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [14]:
#pg4
import pandas as pd
from sqlalchemy import create_engine

def get_engine():
    user = "studente_dapt"
    password = "Ep1c0d3!!D4t4**4n4lys1s"
    host = "epicode-data-pt-mysql.cvetyjye2qbl.eu-central-1.rds.amazonaws.com"
    port = 3306
    db   = "AdventureWorksDW"

    uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db}"
    engine = create_engine(uri, echo=False)   # echo=False disables verbose SQL logging
    return engine

engine = get_engine()          # keep the engine alive for the whole cell

query = """
SELECT
    CustomerKey,
    FirstName,
    LastName,
    EmailAddress,
    Phone,
    AddressLine1
FROM dimcustomer
"""
df = pd.read_sql(query, engine)

print("🔹 Prime 5 righe del DataFrame originale")
print(df.head())

# -----------------------------------------------------------------
# 1. Nomi in minuscolo, cognomi in MAIUSCOLO
# -----------------------------------------------------------------
df["FirstName"] = df["FirstName"].str.lower()
df["LastName"]  = df["LastName"].str.upper()

# -----------------------------------------------------------------
# 2. Split EmailAddress → username e dominio
# -----------------------------------------------------------------
email_parts = df["EmailAddress"].str.split("@", expand=True)
df["EmailUser"]   = email_parts[0]   # prima della @
df["EmailDomain"] = email_parts[1]   # dopo la @

# -----------------------------------------------------------------
# 3. Split Phone in componenti (es. "1 (11) 500 555-0162")
# -----------------------------------------------------------------
# La regex \s+ separa per spazi, mantenendo le parentesi intatte
df["PhoneParts"] = df["Phone"].str.split(r"\s+", expand=False)

# -----------------------------------------------------------------
# 4. Filtra e‑mail che contengono “21”
# -----------------------------------------------------------------
mask_21 = df["EmailAddress"].str.contains("21")
emails_con_21 = df.loc[mask_21, "EmailAddress"]

print("\n📧 E‑mail contenenti '21'")
print(emails_con_21.tolist())

# -----------------------------------------------------------------
# 1. E‑mail che contengono “20” oppure “10”
# -----------------------------------------------------------------
mask_20_10 = df["EmailAddress"].str.contains(r"20|10")
emails_20_10 = df.loc[mask_20_10, "EmailAddress"]
print("\n📧 E‑mail contenenti '20' o '10'")
print(emails_20_10.tolist())

# -----------------------------------------------------------------
# 2. Lunghezza di ogni e‑mail → 5 più lunghe e 5 più corte
# -----------------------------------------------------------------
df["EmailLen"] = df["EmailAddress"].str.len()

top5_long  = df.nlargest(5, "EmailLen")[["EmailAddress", "EmailLen"]]
top5_short = df.nsmallest(5, "EmailLen")[["EmailAddress", "EmailLen"]]

print("\n🔝 5 e‑mail più lunghe")
print(top5_long.to_string(index=False))

print("\n🔽 5 e‑mail più corte")
print(top5_short.to_string(index=False))

# -----------------------------------------------------------------
# 3. Cambia dominio da "adventure-works.com" a "aw-db.com"
# -----------------------------------------------------------------
df["EmailAddress_Modified"] = df["EmailAddress"].str.replace(
    "adventure-works.com", "aw-db.com", regex=False
)

# -----------------------------------------------------------------
# 4. Estrarre righe di AddressLine1 che contengono "Street"
# -----------------------------------------------------------------
mask_street = df["AddressLine1"].str.contains("Street", case=False, na=False)
addresses_with_street = df.loc[mask_street, ["CustomerID", "AddressLine1"]]

print("\n🏠 Indirizzi con la parola 'Street'")
print(addresses_with_street.to_string(index=False))

🔹 Prime 5 righe del DataFrame originale
   CustomerKey  FirstName LastName                    EmailAddress  \
0        11000        Jon     Yang       jon24@adventure-works.com   
1        11001     Eugene    Huang    eugene10@adventure-works.com   
2        11002      Ruben   Torres     ruben35@adventure-works.com   
3        11003    Christy      Zhu   christy12@adventure-works.com   
4        11004  Elizabeth  Johnson  elizabeth5@adventure-works.com   

                 Phone         AddressLine1  
0  1 (11) 500 555-0162      3761 N. 14th St  
1  1 (11) 500 555-0110           2243 W St.  
2  1 (11) 500 555-0184     5844 Linden Land  
3  1 (11) 500 555-0162     1825 Village Pl.  
4  1 (11) 500 555-0131  7553 Harness Circle  

📧 E‑mail contenenti '21'
['chase21@adventure-works.com', 'alvin21@adventure-works.com', 'caroline21@adventure-works.com', 'gabriel21@adventure-works.com', 'luis21@adventure-works.com', 'allison21@adventure-works.com', 'adrian21@adventure-works.com', 'latasha21@a

KeyError: "['CustomerID'] not in index"

In [4]:
#pg6
import pandas as pd

df = pd.read_csv("C:/Users/matte/Downloads/archive/beginner_datasets/facebook.csv")
print(df.columns)

df['status_published'] = pd.to_datetime(df['status_published'], errors='coerce')
df['year'] = df['status_published'].dt.year
df['month'] = df['status_published'].dt.month
df['day'] = df['status_published'].dt.day
df['dayofweek'] = df['status_published'].dt.dayofweek
df['dayofyear'] = df['status_published'].dt.dayofyear
print(df[['status_published','year','month','day','dayofweek','dayofyear']].head())
print("Valori mancanti in status_published:", df['status_published'].isna().sum())

#Esercizio 2/2
#Req.1
posts_2012 = df[df['year'] == 2012].copy()
print("Numero post 2012:", len(posts_2012))

#Req.2
posts_may_2018 = df[(df['year'] == 2018) & (df['month'] == 5)].copy()
print("Numero post maggio 2018:", len(posts_may_2018))

#Req.3 Definizione weekend: sabato (5) e domenica (6)
df['is_weekend'] = df['dayofweek'].isin([5,6])

counts = df['is_weekend'].value_counts().rename(index={False:'weekday', True:'weekend'})
print(counts)

percent = df['is_weekend'].value_counts(normalize=True).rename(index={False:'weekday', True:'weekend'}) * 100
print(percent.round(2))

#Req.4
df_valid = df.dropna(subset=['status_published']).copy()
df_valid = df_valid.sort_values('status_published') #ordinamento per data

first_per_year = df_valid.groupby('year').first().reset_index()
last_per_year = df_valid.groupby('year').last().reset_index()

#Req.5
type_col = None
for candidate in ['type','status_type','post_type']:
    if candidate in df.columns:
        type_col = candidate
        break

if type_col is None:
    print("Non ho trovato una colonna tipo; controlla i nomi delle colonne:", df.columns)
else:
    unique_types = df[type_col].nunique(dropna=True)
    counts_by_type = df[type_col].value_counts(dropna=False)
    print("Numero tipi distinti:", unique_types)
    print("Conteggio per tipo:")
    print(counts_by_type)


Index(['status_id', 'status_type', 'status_published', 'num_reactions',
       'num_comments', 'num_shares', 'num_likes', 'num_loves', 'num_wows',
       'num_hahas', 'num_sads', 'num_angrys'],
      dtype='object')
     status_published  year  month  day  dayofweek  dayofyear
0 2018-04-22 06:00:00  2018      4   22          6        112
1 2018-04-21 22:45:00  2018      4   21          5        111
2 2018-04-21 06:17:00  2018      4   21          5        111
3 2018-04-21 02:29:00  2018      4   21          5        111
4 2018-04-18 03:22:00  2018      4   18          2        108
Valori mancanti in status_published: 0
Numero post 2012: 589
Numero post maggio 2018: 595
is_weekend
weekday    5009
weekend    2041
Name: count, dtype: int64
is_weekend
weekday    71.05
weekend    28.95
Name: proportion, dtype: float64
Numero tipi distinti: 4
Conteggio per tipo:
status_type
photo     4288
video     2334
status     365
link        63
Name: count, dtype: int64


In [7]:
#pg8
import pandas as pd

df = pd.read_csv("C:/Users/matte/Downloads/archive/beginner_datasets/pokemon.csv")

null_counts = df.isnull().sum()
print("Valori nulli per colonna:")
print(null_counts)

rows_with_null = df.isnull().any(axis=1).sum()
print("\nRighe con almeno un valore nullo:", rows_with_null)

print("\nPrime righe con valori nulli:")
print(df[df.isnull().any(axis=1)].head(10))

df_dropped_specific = df.dropna(subset=['Name','#']).reset_index(drop=True)

Valori nulli per colonna:
#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

Righe con almeno un valore nullo: 386

Prime righe con valori nulli:
     #                     Name  Type 1 Type 2  Total  HP  Attack  Defense  \
4    4               Charmander    Fire    NaN    309  39      52       43   
5    5               Charmeleon    Fire    NaN    405  58      64       58   
9    7                 Squirtle   Water    NaN    314  44      48       65   
10   8                Wartortle   Water    NaN    405  59      63       80   
11   9                Blastoise   Water    NaN    530  79      83      100   
12   9  BlastoiseMega Blastoise   Water    NaN    630  79     103      120   
13  10                 Caterpie     Bug    NaN    195  45      30       35   
14  11                  Metapod  

In [8]:
#pg9
import pandas as pd

df = pd.read_csv("C:/Users/matte/Downloads/archive/beginner_datasets/automobile.csv")

null_counts = df.isnull().sum()
print("Valori nulli per colonna:")
print(null_counts)

rows_with_any_null = df.isnull().any(axis=1).sum()
print("\nRighe con almeno un valore nullo:", rows_with_any_null)

null_doors = df[df['num-of-doors'].isnull()].copy()
print("\nNumero righe con num-of-doors mancante:", len(null_doors))
print(null_doors.head(20))


Valori nulli per colonna:
symboling             0
normalized-losses    37
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  0
stroke                0
compression-ratio     0
horsepower            0
peak-rpm              0
city-mpg              0
highway-mpg           0
price                 0
dtype: int64

Righe con almeno un valore nullo: 38

Numero righe con num-of-doors mancante: 2
    symboling  normalized-losses   make fuel-type aspiration num-of-doors  \
26          1              148.0  dodge       gas      turbo          NaN   
60          0                NaN  mazda    diesel        std          NaN   

   body-style drive-whee

In [9]:
#pg10
import numpy as np
import pandas as pd

temp = pd.DataFrame({
    "Giorno": [0,1,2,3,4,5,6,7,8,9,10,11,12],
    "Temperature": [18,19,18,np.nan,21,20,20,np.nan,21,23,np.nan,23,24]
})

print(temp.isnull().sum())  
print(temp[temp['Temperature'].isna()])  

Giorno         0
Temperature    3
dtype: int64
    Giorno  Temperature
3        3          NaN
7        7          NaN
10      10          NaN


In [11]:
#pg11
import os
import pandas as pd

datasets_dir = "C:/Users/matte/Downloads/archive/beginner_datasets/"   # modifica se serve

files_with_nulls = []
summary = []

for fname in os.listdir(datasets_dir):
    if not fname.lower().endswith(".csv"):
        continue

    fullpath = os.path.join(datasets_dir, fname)
    try:
        df = pd.read_csv(fullpath)
    except Exception as e:
        print(f"Errore leggendo {fname}: {e}")
        continue

    null_counts = df.isna().sum()
    total_nulls = int(null_counts.sum())

    if total_nulls > 0:
        files_with_nulls.append(fname)
        summary.append({
            "file": fname,
            "total_nulls": total_nulls,
            "nulls_per_column": null_counts.to_dict()
        })

print("File CSV che contengono valori nulli:")
for s in summary:
    print(f"- {s['file']}: {s['total_nulls']} null totali")
    for col, cnt in s['nulls_per_column'].items():
        if cnt > 0:
            print(f"    {col}: {cnt}")

File CSV che contengono valori nulli:
- automobile.csv: 39 null totali
    normalized-losses: 37
    num-of-doors: 2
- elections.csv: 52 null totali
    state_abbreviation: 52
- france.csv: 66 null totali
    CustomerID: 66
- hepatitis.csv: 153 null totali
    STEROID: 1
    LIVER BIG: 9
    LIVER FIRM: 10
    SPLEEN PALPABLE: 4
    SPIDERS: 4
    ASCITES: 4
    VARICES: 4
    BILIRUBIN: 5
    ALK PHOSPHATE: 28
    SGOT: 3
    ALBUMIN: 15
    PROTIME: 66
- house.csv: 7829 null totali
    LotFrontage: 259
    Alley: 1369
    MasVnrType: 872
    MasVnrArea: 8
    BsmtQual: 37
    BsmtCond: 37
    BsmtExposure: 38
    BsmtFinType1: 37
    BsmtFinType2: 38
    Electrical: 1
    FireplaceQu: 690
    GarageType: 81
    GarageYrBlt: 81
    GarageFinish: 81
    GarageQual: 81
    GarageCond: 81
    PoolQC: 1453
    Fence: 1179
    MiscFeature: 1406
- income.csv: 4262 null totali
    workclass: 1836
    occupation: 1843
    native-country: 583
- mice.csv: 1396 null totali
    DYRK1A_N: 3
    IT